In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_miocardial()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.40)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (1700, 110)
5.62% missing data
Class distribution: (array([0, 1]), array([1429,  271]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 2),
                       (46, 9, 1),
                       (96, 9, 5),
                       (96, 11, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 5.84 sec - Loss 0.167764 - ACC 67.23% - ACC Mean 67.23% - AUC 72.85% - AUC Mean 72.85% - Deter 000
Ite 00050 - 0.94 sec - Loss 0.067565 - ACC 67.83% - ACC Mean 68.15% - AUC 71.37% - AUC Mean 72.94% - Deter 043
Ite 00057 - 0.89 sec - Loss 0.062088 - ACC 68.12% - ACC Mean 67.89% - AUC 73.01% - AUC Mean 72.66% - Deter 050
Early stop ite 57, rollback to correction of ite 7, whith acc of 69.93% and auc of 76.32%
OUR METHOD RUN 1/10 - acc: 69.93% +- 0.0% - auc: 76.32% +- 0.0%
Ite 00000 - 1.01 sec - Loss 0.183461 - ACC 69.98% - ACC Mean 69.98% - AUC 73.92% - AUC Mean 73.92% - Deter 000
Ite 00050 - 1.02 sec - Loss 0.086285 - ACC 67.49% - ACC Mean 69.55% - AUC 71.71% - AUC Mean 73.71% - Deter 033
Ite 00067 - 0.94 sec - Loss 0.070182 - ACC 69.06% - ACC Mean 68.54% - AUC 73.19% - AUC Mean 72.33% - Deter 050
Early stop ite 67, rollback to correction of ite 17, whith acc of 70.72% and auc of 76.85%
OUR METHOD RUN 2/10 - acc: 70.72% +- 0.0% - auc: 76.85% +- 0.0%
Ite 00000 - 1.08 sec - Lo

## Run Median, Mean and KNN

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

median_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
knn_imputer = KNNImputer(n_neighbors=10, weights='uniform')

median_completed = median_imputer.fit_transform(data_missing_nans)
mean_completed = mean_imputer.fit_transform(data_missing_nans)
knn_completed = knn_imputer.fit_transform(data_missing_nans)

## All runs would be the same since deterministic methods
scores = utils.get_scores(median_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEDIAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
median_accs = acc
median_aucs = auc

scores = utils.get_scores(mean_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MEAN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mean_accs = acc
mean_aucs = auc

scores = utils.get_scores(knn_completed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'KNN - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
knn_accs = acc
knn_aucs = auc

MEDIAN - acc: 56.82% +- 0.0% - auc: 61.13% +- 0.0%
MEAN - acc: 71.28% +- 0.0% - auc: 74.72% +- 0.0%
KNN - acc: 67.56% +- 0.0% - auc: 71.19% +- 0.0%


## Run GAIN

In [6]:
gain_parameters = {'batch_size': 128,
                   'hint_rate': .9,
                   'alpha': 100,
                   'iterations': 5000}

gain_accs, gain_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = gain(data_missing_nans, gain_parameters)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nGAIN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    gain_accs.append(acc.mean())
    gain_aucs.append(auc.mean())
print(f'GAIN GLOBAL - acc: {round(np.array(gain_accs).mean() * 100, 2)}% +- {round(np.array(gain_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(gain_aucs).mean() * 100, 2)}% +- {round(np.array(gain_aucs).std() * 100, 2)}%')

100%|██████████| 5000/5000 [00:47<00:00, 104.66it/s]



GAIN RUN 1/10 - acc: 68.49% +- 1.76% - auc: 70.9% +- 2.89%



100%|██████████| 5000/5000 [00:49<00:00, 101.23it/s]



GAIN RUN 2/10 - acc: 63.85% +- 2.03% - auc: 69.14% +- 1.75%



100%|██████████| 5000/5000 [00:48<00:00, 102.74it/s]


...


GAIN RUN 8/10 - acc: 63.9% +- 2.15% - auc: 70.4% +- 1.74%



100%|██████████| 5000/5000 [01:00<00:00, 82.73it/s]



GAIN RUN 9/10 - acc: 66.52% +- 3.7% - auc: 71.4% +- 2.94%



100%|██████████| 5000/5000 [01:05<00:00, 76.28it/s]



GAIN RUN 10/10 - acc: 65.98% +- 1.77% - auc: 69.52% +- 3.54%

GAIN GLOBAL - acc: 66.13% +- 2.16% - auc: 70.44% +- 1.62%


## Run MICE

In [7]:
imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 72.06% +- 0.0% - auc: 74.75% +- 0.0%


## Run MISSFOREST

In [8]:
miss_accs, miss_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = MissForest(n_jobs=-1, random_state=i)
    imputed = imputer.fit_transform(data_missing_nans)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMISSFOREST RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    miss_accs.append(acc.mean())
    miss_aucs.append(auc.mean())
print(f'\nMISSFOREST GLOBAL - acc: {round(np.array(miss_accs).mean() * 100, 2)}% +- {round(np.array(miss_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(miss_aucs).mean() * 100, 2)}% +- {round(np.array(miss_aucs).std() * 100, 2)}%')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 1/10 - acc: 70.97% +- 1.14% - auc: 73.17% +- 2.05%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 2/10 - acc: 69.48% +- 3.42% - auc: 73.74% +- 2.73%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 3/10 - acc: 69.58% +- 2.61% - auc: 73.54% +- 2.27%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 4/10 - acc: 69.8% +- 2.16% - auc: 72.29% +- 2.6%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3

MISSFOREST RUN 5/10 - acc: 70.61% +- 2.67% - auc: 73.29% +- 2.68%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6

MISSFOREST RUN 6/10 - acc: 69.5% +- 4.18% - auc: 72.39% +- 4.15%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5

MISSFOREST RUN 7/10 - acc: 68.65% +- 3.86% - auc: 74.0% +- 3.29%

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4

MISSFOR

## Run SOFTIMPUTE

In [9]:
soft_accs, soft_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = SoftImpute(random_state=i)
    imputer.fit(data_missing_nans)
    imputed = imputer.predict(data_missing_nans)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'SOFTIMPUTE RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    soft_accs.append(acc.mean())
    soft_aucs.append(auc.mean())
print(f'\nSOFTIMPUTE GLOBAL - acc: {round(np.array(soft_accs).mean() * 100, 2)}% +- {round(np.array(soft_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(soft_aucs).mean() * 100, 2)}% +- {round(np.array(soft_aucs).std() * 100, 2)}%')

SOFTIMPUTE RUN 1/10 - acc: 70.69% +- 3.04% - auc: 75.18% +- 3.71%
SOFTIMPUTE RUN 2/10 - acc: 70.39% +- 2.91% - auc: 73.33% +- 4.15%
SOFTIMPUTE RUN 3/10 - acc: 70.52% +- 3.46% - auc: 73.3% +- 4.31%
SOFTIMPUTE RUN 4/10 - acc: 70.03% +- 3.93% - auc: 75.98% +- 3.02%
SOFTIMPUTE RUN 5/10 - acc: 70.36% +- 3.27% - auc: 72.63% +- 5.52%
SOFTIMPUTE RUN 6/10 - acc: 70.56% +- 2.99% - auc: 73.42% +- 4.46%
SOFTIMPUTE RUN 7/10 - acc: 71.02% +- 2.8% - auc: 73.08% +- 5.64%
SOFTIMPUTE RUN 8/10 - acc: 71.85% +- 3.73% - auc: 74.2% +- 5.15%
SOFTIMPUTE RUN 9/10 - acc: 71.11% +- 3.16% - auc: 72.77% +- 5.16%
SOFTIMPUTE RUN 10/10 - acc: 71.49% +- 2.6% - auc: 74.18% +- 5.7%

SOFTIMPUTE GLOBAL - acc: 70.8% +- 0.53% - auc: 73.81% +- 1.03%


## Run SINKHORN

In [10]:
sink_accs, sink_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputer = OTimputer(niter=500)
    imputed = imputer.fit_transform(data_missing_nans).cpu().detach().numpy()
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nSINKHORN RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    sink_accs.append(acc.mean())
    sink_aucs.append(auc.mean())
print(f'\nSINKHORN GLOBAL - acc: {round(np.array(sink_accs).mean() * 100, 2)}% +- {round(np.array(sink_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(sink_aucs).mean() * 100, 2)}% +- {round(np.array(sink_aucs).std() * 100, 2)}%')

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 7.7756

SINKHORN RUN 1/10 - acc: 66.31% +- 2.63% - auc: 70.03% +- 2.8%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 7.7849

SINKHORN RUN 2/10 - acc: 69.02% +- 4.01% - auc: 71.67% +- 3.09%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 7.8881

SINKHORN RUN 3/10 - acc: 69.17% +- 3.74% - auc: 71.0% +- 3.84%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 8.0223

SINKHORN RUN 4/10 - acc: 68.71% +- 5.22% - auc: 70.47% +- 5.01%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 7.8125

SINKHORN RUN 5/10 - acc: 67.88% +- 3.66% - auc: 71.84% +- 3.41%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 7.7409

SINKHORN RUN 6/10 - acc: 70.66% +- 2.05% - auc: 73.79% +- 1.0%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 7.9821

SINKHORN RUN 7/10 - acc: 68.78% +- 3.47% - auc: 73.56% +- 2.99%

batchsize = 128, epsilon = 0.0100
Iteration 0:	 Loss: 7.9311

SINKHORN RUN 8/10 - acc: 68.84% +- 1.11% - auc: 70.7

## Run MIDA

In [11]:
mida_accs, mida_aucs = [], []
for i in range(n_runs):
    random.seed(i)
    np.random.seed(i)
    torch.manual_seed(i)
    imputed = mida(data_missing_nans, num_layers=2, num_epochs=1500)
    imputed = np.where(missing_mask, data_missing, imputed)
    scores = utils.get_scores(imputed, y)
    acc = scores['test_balanced_accuracy']
    auc = scores['test_roc_auc_ovo']
    print(f'\nMIDA RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%\n')
    mida_accs.append(acc.mean())
    mida_aucs.append(auc.mean())
print(f'\nMIDA GLOBAL - acc: {round(np.array(mida_accs).mean() * 100, 2)}% +- {round(np.array(mida_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(mida_aucs).mean() * 100, 2)}% +- {round(np.array(mida_aucs).std() * 100, 2)}%')

Stop training at epoch: 1500/1500, return best output

MIDA RUN 1/10 - acc: 69.51% +- 1.86% - auc: 71.4% +- 3.53%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 2/10 - acc: 70.05% +- 4.2% - auc: 72.54% +- 3.34%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 3/10 - acc: 69.12% +- 1.12% - auc: 71.87% +- 2.02%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 4/10 - acc: 66.73% +- 2.3% - auc: 70.39% +- 3.56%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 5/10 - acc: 69.58% +- 1.36% - auc: 71.36% +- 3.06%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 6/10 - acc: 68.52% +- 2.01% - auc: 71.23% +- 2.5%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 7/10 - acc: 68.06% +- 3.56% - auc: 70.08% +- 5.04%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 8/10 - acc: 67.71% +- 2.73% - auc: 69.44% +- 2.23%

Stop training at epoch: 1500/1500, return best output

MIDA RUN 9/10 - acc: 

## Run T-tests

In [12]:
for model, metrics in {
        'MEDIAN': {'ACC': median_accs, 'AUC': median_aucs},
        'MEAN': {'ACC': mean_accs, 'AUC': mean_aucs},
        'KNN': {'ACC': knn_accs, 'AUC': knn_aucs},
        'GAIN': {'ACC': gain_accs, 'AUC': gain_aucs},
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'MISS': {'ACC': miss_accs, 'AUC': miss_aucs},
        'SOFT': {'ACC': soft_accs, 'AUC': soft_aucs},
        'SINK': {'ACC': sink_accs, 'AUC': sink_aucs},
        'MIDA': {'ACC': mida_accs, 'AUC': mida_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MEDIAN
Metric AUC - OUR METHOD is significantly better than MEDIAN
Metric ACC - OUR METHOD is significantly worse than MEAN
Metric AUC - OUR METHOD is significantly better than MEAN
Metric ACC - OUR METHOD is significantly better than KNN
Metric AUC - OUR METHOD is significantly better than KNN
Metric ACC - OUR METHOD is significantly better than GAIN
Metric AUC - OUR METHOD is significantly better than GAIN
Metric ACC - OUR METHOD is significantly worse than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is even with MISS
Metric AUC - OUR METHOD is significantly better than MISS
Metric ACC - OUR METHOD is even with SOFT
Metric AUC - OUR METHOD is significantly better than SOFT
Metric ACC - OUR METHOD is significantly better than SINK
Metric AUC - OUR METHOD is significantly better than SINK
Metric ACC - OUR METHOD is significantly better than MIDA
Metric AUC - OUR METHOD is significantly bette